# Generating 26 cancer gene regulatory network using TCGA datasets
Authors: Camila Lopes-Ramos<sup>1</sup>

<sup>1</sup> Department of Biostatistics, Harvard T.H. Chan School of Public Health, Boston, MA.

## Introduction

In this tutorial, we will generate 26 cancer gene regulatory networks using PANDA<sup>1</sup> from TCGA gene expression data. First, we need to normalize gene expression data accounting for batch effects using smooth quantile normalization (qsmooth)<sup>2</sup>, then we will call PANDA by using gene expression for each tissue as input along with TF PPI from StringDB<sup>3</sup>, and a motif prior network using TF motifs from CIS-BP ran on promoter regions of target genes using FIMO<sup>4</sup>.

The generated networks are available in the cancer catalog of [GRAND](https://grand.networkmedicine.org/) database.

In [ ]:
library(Biobase)
library(qsmooth)# for quantile normalization
library(netZooR) # for GRN reconstruction using PANDA

## 1. Normalize gene expression data using qsmooth
This script normalizes the raw counts by qsmooth<sup>2</sup>, log2 transform and save one expression file for each of 26 cancer types

In [ ]:
load("/opt/data/netZooR/tcganets/rnaseq_tumor_clin2019_06_26.rdata")

Qsmooth normalization

In [ ]:
counts <- exprs(obj2)
qsmooth1 <- qsmooth(object = counts, group_factor = pData(obj2)$tumorID)
qs <- qsmoothData(qsmooth1)
exp <- log2(qs+1)

Save clinical and expression files for each tumor type (log2 transformed)

In [ ]:
cancer <- unique(pData(obj2)$tumorID)
for(i in 1:length(cancer)){
  k <- which(pData(obj2)$tumorID == cancer[i])
  exp1 <- exp[,k]
  clin <- pData(obj2)[k,]
  d <- ifelse(is.na(clin$days_to_last_followup), yes=clin$last_contact_days_to, no=clin$days_to_last_followup)
  clin$days_to_last_followup <- d
  j <- c("age_at_initial_pathologic_diagnosis", "gender", "race", "ajcc_pathologic_tumor_stage", "days_to_last_followup", "vital_status", "tumorID")
  clin1 <- clin[,which(colnames(clin) %in% j)]
  
  write.table(exp1, file=paste0("expression/expression_tcga_", cancer[i],".txt"),sep="\t",quote=F)
  write.table(clin1,file=paste0("phenotypic/pheno_tcga_", cancer[i],".txt"),sep="\t",quote=F)
}

## 2. Gene regulatory network reconstruction
Now, we will generate 26 cancer networks using the data processed above and PANDA<sup>1</sup> as implemented in [netZooR](netzoo.github.io) v0.9.

In [ ]:
for (cancer in c("ACC","BLCA","CHOL","COAD","DLBC","ESCA","GBM","HNSC","KICH","KIRC","KIRP","LAML","LGG","LIHC","LUAD","LUSC","MESO","PAAD","PCPG","READ","SARC","SKCM","STAD","THCA","THYM","UVM")){
  exp <- read.delim(paste0("expression/expression_tcga_",cancer,".txt"), check.names = FALSE)
  ppi <- read.delim("prior/ppi.txt",stringsAsFactors=F,header=F)
  motif <- read.delim("prior/motif.txt",stringsAsFactors=F,header=F)
  
  
  panda_results <- panda(motif,exp, ppi, mode="intersection")
  write.csv(panda_results@regNet, file=paste0("network/panda_", cancer, ".csv"), quote=F)
}

## References
1 - Glass, Kimberly, et al. "Passing messages between biological networks to refine predicted interactions." PloS one 8.5 (2013): e64832.

2 - Hicks, Stephanie C., et al. "Smooth quantile normalization." Biostatistics 19.2 (2018): 185-198.

3 - Szklarczyk, Damian, et al. "The STRING database in 2017: quality-controlled protein–protein association networks, made broadly accessible." Nucleic acids research (2016): gkw937.

4 - Weirauch, Matthew T., et al. "Determination and inference of eukaryotic transcription factor sequence specificity." Cell 158.6 (2014): 1431-1443.